In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_N.pickle to train_N.pickle


In [ ]:
ls

Epi-16256.pth  sample_data/


In [ ]:
!pip install tensorboardX

In [ ]:
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir=/content/runs/20230116

In [ ]:
#!tar -cvf data.tar data/
!gzip data.tar
#!tar zxvf data.tar.gz #解壓縮(須先改名)
#!unzip data.zip

##################   My   Code   ###########################

In [ ]:
import numpy as np
import pandas as pd
import pickle

from collections import Counter
from tensorboardX import SummaryWriter

with open('train_N.pickle','rb') as f:
    data = pickle.load(file = f)#data=> list
dataN = len(data)

df = pd.DataFrame(data, columns =['Price', 'Symb'])
df.insert(2, "dailyR", np.nan)
for i in range(1,dataN):
  if df['Symb'][i][3:5] == df['Symb'][i-1][3:5]:
    df.at[i, 'dailyR'] = np.log(df['Price'][i].iloc[283,1])-np.log(df['Price'][i-1].iloc[283,1])

clP = np.zeros(dataN)
for j in range(dataN):
  clP[j] = df['Price'][j].iloc[283,1]#每日13:29的價
pM = clP.mean()
pS = clP.std()
clPNorm =  (clP - pM) / pS

class Reporter:
    def __init__(self, report_interval: int = 1):
        self.counter = Counter()
        self.graph_initialized = False
        self.report_interval = report_interval
        self.t = 0
        self.writer = SummaryWriter('runs/20230116')#

    def will_report(self, tag: str) -> bool:
        return self.counter[tag] % (self.report_interval + 1) == 0

    def scalar(self, tag: str, value: float):
        if self.will_report(tag):
            self._scalar(tag, value, self.counter[tag])
        self.counter[tag] += 1

    def graph(self, model, input_to_model):
        if not self.graph_initialized:
            self._graph(model, input_to_model)
            self.graph_initialized = True

    def _scalar(self, tag: str, value: float, step: int):
        self.writer.add_scalar(tag, value, step)

class observation_space:
    def __init__(self, n):
        self.shape = (n,)

class action_space:
    def __init__(self, n):
        self.n = n
    def seed(self, seed):
        pass
    def sample(self):
        return random.randint(0, self.n - 1)

class Finance:
    def __init__(self, features, window, lags):
        self.features = features
        self.observation_space = observation_space(len(features))
        self.action_space = action_space(3)#ori:2  ????????????????? 
        self.window = window#如5日均值
        self.lags = lags #相當於batach_size
        self.osn = self.lags       
        self.min_accuracy = 0.475
        self._prepare_data()

    def _prepare_data(self):
        self.data = df
        self.data['r'] = df['dailyR']
        self.data.dropna(inplace=True)       
        #self.data['d'] = np.where(self.data['r'] > 0, 1, 0) #ori
        self.data['d'] = np.where(self.data['r'] > 0, 1, 2)
        self.data['clPNorm'] = pd.DataFrame(clPNorm)
        self.data['rNorm'] = (self.data['r']-self.data['r'].mean())/self.data['r'].std()
        self.data['clPNormAvg'] = self.data['clPNorm'].rolling(self.window).mean()
        self.data['clPNormStd'] = self.data['clPNorm'].rolling(self.window).std()
        self.data['rNormAvg'] = self.data['rNorm'].rolling(self.window).mean()
        self.data['rNormStd'] = self.data['rNorm'].rolling(self.window).std()
        self.data.dropna(inplace=True)#這行會使平均值,標準差的NaN都消失!!!!!!!!!!!!!!!!

    def _get_state(self):
        return self.data[self.features].iloc[self.bar-self.lags : self.bar]

    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.bar = self.osn#必須先要reset才會有self.bar, 使用step(a)才不會出錯
        state = self.data[self.features].iloc[self.bar-self.osn : self.bar]#會按照順序
        return state.values

    def step(self, action):
        correct = action == self.data['d'].iloc[self.bar]
        #ret = self.data['rNorm'].iloc[self.bar]
        reward_1 = 1 if correct else 0
        self.treward += reward_1
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.osn)

        if self.bar >= len(learn_env.data):
            done = True
        elif reward_1 == 1:
            done = False
        elif ((self.accuracy < self.min_accuracy) and (self.bar > self.osn + 10)):# Xa.ipynb
            done = True
        else:
            done = False
      
        state = self._get_state()
        info = {}

        return state.values, reward_1, done, info

#features = ['clPNorm','rNorm']
features = ['clPNorm','rNorm','clPNormAvg','clPNormStd','rNormAvg','rNormStd']
window = 5 #平均值時會用到
lags = 4
learn_env = Finance(features, window, lags)#

"""
(self, features, window, lags)
learn_env.reset()
#s = learn_env.reset()

clP: 590個日收盤價
learn_env.data['rNorm'].iloc[0] => nan
learn_env.data.head(10)
learn_env.data['dailyR'][14:22]
"""

"\n(self, features, window, lags)\nlearn_env.reset()\n#s = learn_env.reset()\n\nclP: 590個日收盤價\nlearn_env.data['rNorm'].iloc[0] => nan\nlearn_env.data.head(10)\nlearn_env.data['dailyR'][14:22]\n"

In [ ]:
import random
import torch
import torch.nn as nn
from torch.distributions import Categorical
from torch.optim import Adam
import torch.nn.functional as F
from collections import deque
import os

class CategoricalPolicy(nn.Module):
    def __init__(self, hidden_size, state_dim=6, action_dim=3):#
        super(CategoricalPolicy, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim

        ######################## Q1 Head ##################
        # Construct Hidden Layer 1 with state
        self.f1 = nn.Linear(state_dim, hidden_size)

        #Hidden Layer 2
        self.f2 = nn.Linear(hidden_size, hidden_size)

        #Value
        self.val = nn.Linear(hidden_size, 1)

        #Advantages
        self.adv = nn.Linear(hidden_size, action_dim)



    def clean_action(self, obs, return_only_action=True):
        ###### Feature ####
        info = torch.relu(self.f1(obs))#RuntimeError: mat1 and mat2 must have the same dtype
        info = torch.relu(self.f2(info))

        val = self.val(info)
        adv = self.adv(info)

        logits = val + adv - adv.mean()

        if return_only_action:
            return logits.argmax(1)

        return None, None, logits

    def noisy_action(self, obs, return_only_action=True):
        _, _, logits = self.clean_action(obs, return_only_action=False)
        dist = Categorical(logits=logits)
        action = dist.sample()
        action = action

        if return_only_action:
            return action

        return action, None, logits

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(param.data)

def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(target_param.data * (1.0 - tau) + param.data * tau)

class FQLAgent:
    def __init__(self, hidden_dim, learn_env, learning_rate=0.0001, alpha=0.1, tau=1e-3):
        self.learn_env = learn_env
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.98
        self.learning_rate = learning_rate
        self.gamma = 0.95
        self.alpha = alpha #Alpha for Entropy term
        self.tau = tau #soft_update param. 
        self.batch_size = 128
        self.max_treward = 0
        self.trewards = list()
        self.averages = list()
        self.performances = list()
        self.memory = deque(maxlen=2000)
        self.model = CategoricalPolicy(hidden_dim)
        self.model_target = CategoricalPolicy(hidden_dim)
        hard_update(self.model_target, self.model)
        self.model_optim = Adam(self.model.parameters(), lr=self.learning_rate)
        self.reporter = Reporter()

    def act(self, state):
        state = torch.tensor(state, dtype=torch.float32)
        """      
        if random.random() <= self.epsilon:
            return self.learn_env.action_space.sample()
        """            
        #action, _, _ = self.model.noisy_action(state, return_only_action=False)
        action = self.model.noisy_action(state, return_only_action=True)

        #return torch.argmax(action)
        return action[0]


    def replay(self):
        batch = random.sample(self.memory, self.batch_size)
 
        for state, action, reward, next_state, done in batch:
          state = torch.tensor(state, dtype=torch.float32)
          next_state = torch.tensor(next_state, dtype=torch.float32)
          action = action * torch.ones((1,self.model.action_dim), dtype=torch.int64)#My Code-將action從單一數字改維度,才能作gather計算
          d = 1 if done else 0

          with torch.no_grad():
            na = self.model.clean_action(next_state,  return_only_action=True)
            _, _, ns_logits = self.model_target.noisy_action(next_state, return_only_action=False)
            next_entropy = -(F.softmax(ns_logits, dim=1) * F.log_softmax(ns_logits, dim=1)).mean(1).unsqueeze(1)
            ns_logits = ns_logits.gather(1, na.unsqueeze(1))#20221114 取出action相應的對數機率
            next_target = ns_logits + self.alpha * next_entropy
            next_q_value = reward + (1-d) * self.gamma * next_target

        _, _, logits  = self.model.noisy_action(state, return_only_action=False)
        entropy = -(F.softmax(logits, dim=1) * F.log_softmax(logits, dim=1)).mean(1).unsqueeze(1)
        q_val = logits.gather(1, action)

        q_loss = (next_q_value - q_val)**2
        q_loss -= self.alpha*entropy
        q_loss = q_loss.mean()

        self.model_optim.zero_grad()
        q_loss.backward()
        torch.nn.utils.clip_grad_norm_(parameters=self.model.parameters(), max_norm=10)#
        self.model_optim.step()

        soft_update(self.model_target, self.model, self.tau)

        self.reporter.scalar('q_loss/total', q_loss)

    def learn(self, episodes):
        os.makedirs('./data/', exist_ok=True)
        for e in range(1, episodes + 1):
            state = self.learn_env.reset()
            rewardT = 0
            for _ in range(10000):
                action = self.act(state)
                next_state, reward, done, info = self.learn_env.step(action)
                rewardT += reward
                self.memory.append([state, action, reward, next_state, done])
              
                state = next_state
                if done:
                    treward = _ + 1
                    self.trewards.append(treward)
                    av = sum(self.trewards[-len(self.trewards):]) / len(self.trewards)
                    perf = self.learn_env.accuracy
                    self.averages.append(av)
                    self.performances.append(perf)
                    self.max_treward = max(self.max_treward, treward)
                    templ = 'episode: {:2d}/{} | treward: {:4d} | '
                    templ += 'perf: {:5.3f} | av: {:5.1f} | max: {:4d}'
                    print(templ.format(e, episodes, treward, perf, av, self.max_treward))
                    #if treward > 500:
                    if rewardT > 270:
                      f = open("./data/logfile.txt","a")
                      f.write('Eposode: %d\t' % e)
                      f.write("treward: %d\t" % treward)
                      f.write("perf: %d\t" % perf)
                      f.write("av: %d\t" % av)
                      f.write("rewardT: %d\t" % rewardT)
                      f.write("max: %d\t" % self.max_treward)
                      f.write("\n")
                      f.close()
                      fileN = './data/Epi-'+str(e)+'.pth'
                      torch.save(self.model.state_dict(),fileN)

                    break
            if len(self.memory) > self.batch_size:
                self.replay()
"""
for i in range(1,dataN):
  if data[i][1][3:5] == data[i-1][1][3:5]:
    ret = np.log(data[i][0].iloc[283,1])-np.log(data[i-1][0].iloc[283,1])
    print('i: ',i,' ',ret)
"""

"\nfor i in range(1,dataN):\n  if data[i][1][3:5] == data[i-1][1][3:5]:\n    ret = np.log(data[i][0].iloc[283,1])-np.log(data[i-1][0].iloc[283,1])\n    print('i: ',i,' ',ret)\n"

In [ ]:
agent = FQLAgent(256, learn_env)

In [ ]:
%time agent.learn(5)

episode:  1/5 | treward:   11 | perf: 0.182 | av:  11.0 | max:   11
episode:  2/5 | treward:   12 | perf: 0.417 | av:  11.5 | max:   12
episode:  3/5 | treward:   12 | perf: 0.250 | av:  11.7 | max:   12
episode:  4/5 | treward:   11 | perf: 0.364 | av:  11.5 | max:   12
episode:  5/5 | treward:   12 | perf: 0.167 | av:  11.6 | max:   12
CPU times: user 88 ms, sys: 6.77 ms, total: 94.8 ms
Wall time: 213 ms


In [ ]:
torch.save(agent.model.state_dict(),'./data/Epi-5.pth')

########################### CHECK #################################

In [ ]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn

with open('train_N.pickle','rb') as f:
    data = pickle.load(file = f)#data=> list
dataN = len(data)

df = pd.DataFrame(data, columns =['Price', 'Symb'])
df.insert(2, "dailyR", np.nan)
for i in range(1,dataN):
  if df['Symb'][i][3:5] == df['Symb'][i-1][3:5]:
    df.at[i, 'dailyR'] = np.log(df['Price'][i].iloc[283,1])-np.log(df['Price'][i-1].iloc[283,1])

clP = np.zeros(dataN)
for j in range(dataN):
  clP[j] = df['Price'][j].iloc[283,1]#每日13:29的價
pM = clP.mean()
pS = clP.std()
clPNorm =  (clP - pM) / pS

class observation_space:
    def __init__(self, n):
        self.shape = (n,)

class action_space:
    def __init__(self, n):
        self.n = n
    def seed(self, seed):
        pass
    def sample(self):
        return random.randint(0, self.n - 1)

class Finance:
    def __init__(self, features, window, lags):
        self.features = features
        self.observation_space = observation_space(len(features))
        self.action_space = action_space(3)#ori:2  
        self.window = window#如5日均值
        self.lags = lags #相當於batach_size
        self.osn = self.lags       
        self.min_accuracy = 0.475
        self._prepare_data()

    def _prepare_data(self):
        self.data = df
        self.data['r'] = df['dailyR']
        self.data.dropna(inplace=True)       
        self.data['d'] = np.where(self.data['r'] > 0, 1, 2)
        self.data['clPNorm'] = pd.DataFrame(clPNorm)
        self.data['rNorm'] = (self.data['r']-self.data['r'].mean())/self.data['r'].std()
        self.data['clPNormAvg'] = self.data['clPNorm'].rolling(self.window).mean()
        self.data['clPNormStd'] = self.data['clPNorm'].rolling(self.window).std()
        self.data['rNormAvg'] = self.data['rNorm'].rolling(self.window).mean()
        self.data['rNormStd'] = self.data['rNorm'].rolling(self.window).std()
        self.data.dropna(inplace=True)#這行會使平均值,標準差的NaN都消失

    def _get_state(self):
        return self.data[self.features].iloc[self.bar-self.lags : self.bar]

    def reset(self):
        self.treward = 0
        self.accuracy = 0
        self.bar = self.osn#必須先要reset才會有self.bar, 使用step(a)才不會出錯
        state = self.data[self.features].iloc[self.bar-self.osn : self.bar]#會按照順序
        return state.values

    def step(self, action):
        correct = action == self.data['d'].iloc[self.bar]
        reward_1 = 1 if correct else 0
        self.treward += reward_1
        self.bar += 1
        self.accuracy = self.treward / (self.bar - self.osn)

        if self.bar >= len(self.data):
            done = True
        elif reward_1 == 1:
            done = False
        elif ((self.accuracy < self.min_accuracy) and (self.bar > self.osn + 10)):# Xa.ipynb
            done = True
        else:
            done = False
      
        state = self._get_state()
        info = {}

        return state.values, reward_1, done, info

class CategoricalPolicy(nn.Module):
    def __init__(self, hidden_size, state_dim=6, action_dim=3):#
        super(CategoricalPolicy, self).__init__()
        self.state_dim = state_dim
        self.action_dim = action_dim

        ######################## Q1 Head ##################
        # Construct Hidden Layer 1 with state
        self.f1 = nn.Linear(state_dim, hidden_size)

        #Hidden Layer 2
        self.f2 = nn.Linear(hidden_size, hidden_size)

        #Value
        self.val = nn.Linear(hidden_size, 1)

        #Advantages
        self.adv = nn.Linear(hidden_size, action_dim)



    def clean_action(self, obs, return_only_action=True):
        ###### Feature ####
        info = torch.relu(self.f1(obs))#RuntimeError: mat1 and mat2 must have the same dtype
        info = torch.relu(self.f2(info))

        val = self.val(info)
        adv = self.adv(info)

        logits = val + adv - adv.mean()

        if return_only_action:
            return logits.argmax(1)

        return None, None, logits

    def noisy_action(self, obs, return_only_action=True):
        _, _, logits = self.clean_action(obs, return_only_action=False)
        dist = Categorical(logits=logits)
        action = dist.sample()
        action = action

        if return_only_action:
            return action

        return action, None, logits

    def act(self, state):
        state = torch.tensor(state, dtype=torch.float32)
        
        #action, _, _ = self.model.noisy_action(state, return_only_action=False)
        action = self.clean_action(state, return_only_action=True)
        #print('aaaaaaaaaaaaaaa  ',action)

        #return torch.argmax(action)
        return action[0]

features = ['clPNorm','rNorm','clPNormAvg','clPNormStd','rNormAvg','rNormStd']
window = 5 #平均值時會用到
lags = 4
num_hidden = 256
test_env = Finance(features, window, lags)#

policy_net = CategoricalPolicy(num_hidden)
#state_dict = torch.load('/content/Epi-42689.pth')
state_dict = torch.load('/content/data/Epi-49514.pth')
policy_net.load_state_dict(state_dict)

test_episodes = 50
trewards = []
for e in range(test_episodes):
  state = test_env.reset()
  rewardT = 0
  for _ in range(10000):
    #print('sssssssssssss  ',state)
    action = policy_net.act(state)
    next_state, reward, done, info = test_env.step(action)
    #print('rrrrrrrrrrrrrrr  ',reward,'   ddddddddddddd  ', done)
    rewardT += reward
    state = next_state

    if done:
      treward = _ + 1
      trewards.append(treward)
      av = sum(trewards[-len(trewards):]) / len(trewards)
      print('trewardtreward: ',treward,' rewardT: ',rewardT,' trewardAvg: ',av)
      
      break

    